# Training of MoCoGAN
---
In this Notebook is covered the topic of training the MoCoGAN and the fine tuning of this network.

## Defining global variables and imports
---
In the below cell all of the imports that are needed to the training and some global variables like `device` to use the GPU if available.

In [1]:
import os
import sys
import time
import math
import torch
import skvideo.io
import numpy as np

from glob import glob
from torch.autograd import Variable
from torch import nn, optim, cuda as cu, device, manual_seed

from torch.utils.data import DataLoader
from torchvision.datasets import DatasetFolder
from torchvision.transforms import Lambda, Compose

'''Import variables from train.py'''
img_size = 96
nc = 3
ndf = 64 # from dcgan
ngf = 64
d_E = 10
hidden_size = 100 # guess
d_C = 50
d_M = d_E
nz  = d_C + d_M
criterion = nn.BCELoss()

T = 16 # Hyperparameter for taking #Frames into discriminator.

ngpu       = 1
batch_size = 16
n_iter     = 120000
pre_train  = False

## Addition for training on UCF-101
n_epochs_saveV      = 10
n_epochs_display    = 1
n_epochs_check      = 10
max_frame           = 25
cuda                = True #For compatibility with old version of MocoGan
#### End of additions


seed = 0
manual_seed(seed)
np.random.seed(seed)

device = device("cuda" if cu.is_available() else "cpu")

## Import of Models
From the `models` module, let's import all of the models and let's load the previous state for fine tuning.

Then models are moved into the device chosen in the cell above.

In [2]:
sys.path.append("./mocogan/")
from models import Discriminator_I, Discriminator_V, Generator_I ,GRU

In [3]:
'''Create the objects for Discriminator_(I|V), GRU and Generator_I'''
gen_i = Generator_I(nc, ngf, nz, ngpu = 1)
gru = GRU(d_E, hidden_size, gpu = cu.is_available())
dis_i = Discriminator_I(nc, ndf, ngpu = 1)
dis_v = Discriminator_V(nc, ndf, T = T, ngpu = 1)
gru.initWeight()

'''Move objects into the device chosen'''
''' adjust to cuda '''
if cuda == True:
    dis_i.cuda()
    dis_v.cuda()
    gen_i.cuda()
    gru.cuda()
    criterion.cuda()

'''Optimizer Settings and Optimizer'''
lr = 0.0002
betas=(0.5, 0.999)
optim_Di  = optim.Adam(dis_i.parameters(), lr=lr, betas=betas)
optim_Dv  = optim.Adam(dis_v.parameters(), lr=lr, betas=betas)
optim_Gi  = optim.Adam(gen_i.parameters(), lr=lr, betas=betas)
optim_GRU = optim.Adam(gru.parameters(),   lr=lr, betas=betas)

./mocogan/models.py:155: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  init.xavier_uniform(params)
./mocogan/models.py:165: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(params, 0)


## Dataloader
---
In the cells below, the dataloader will be defined and also all of the transformation that will be applied to the videos before taking them into a batch.


In [4]:
current_path = !pwd
current_path = str(current_path[0])
resized_path = os.path.join(current_path, "mocogan", 'resized_data')
files = glob(resized_path+'/*/*')

#transformation = Compose([ToTensor(), Lambda(lambda tensor: (tensor - tensor.mean() )/ tensor.std())])

transformation = Compose([Lambda(lambda video: video.transpose(3, 0, 1, 2)/255.0),
                            Lambda(lambda video: video[ : , : max_frame, :, : ]),
                            Lambda(lambda video: torch.FloatTensor(video))])

filenameDictClassesIdx = "classInd.txt"
dictClassesIdx = {}
try:
    with open(os.path.join(current_path, "ucfTrainTestlist", filenameDictClassesIdx)) as file:
        for line in file:
            dictClassesIdx[ line.split() [1]] = int( line.split() [0] )
            
except FileNotFoundError as _:
    with open(os.path.join(current_path, "mocogan", "ucfTrainTestlist", filenameDictClassesIdx)) as file:
        for line in file:
            dictClassesIdx[ line.split() [1]] = int( line.split() [0] )

# dataset = DatasetFolder(resized_path, loadVideo, ["mp4"], transform= transformation)
dataset = DatasetFolder(resized_path, skvideo.io.vread, ["mp4"], transform= transformation)
dataset.class_to_idx = dictClassesIdx


dataloader = DataLoader(dataset, batch_size= batch_size, shuffle= True, num_workers= 8, pin_memory= True, drop_last= True)

In [5]:
import cv2

print( cv2.getBuildInformation() )

def loadVideo(videoName):    
    print(f"[CV2] Loading : {videoName}")
    
    cap = cv2.VideoCapture(videoName)
    cv2.cv.CaptureFromFile()
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frameHeight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"{cap.isOpened()}")

    buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

    fc = 0
    ret = True
    
    print("[CV2] Started filling Buffer.")
    
    while (fc < frameCount and ret):
        print(f"Inside While loop: {fc}")
        ret, buf[fc] = cap.read()
        fc += 1
        
    print("[CV2] End filling Buffer.")

    cap.release()

    #cv2.namedWindow('frame 10')
    #cv2.imshow('frame 10', buf[9])
    
    print("[CV2] Waiting...")

    cv2.waitKey(0)
    
    print(f"[CV2] Ready to return: {buf}")
    
    return buf


General configuration for OpenCV 3.1.0 =====================================
  Version control:               unknown

  Platform:
    Host:                        Linux 4.8.12-040812-generic x86_64
    CMake:                       3.6.3
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               Release

  C/C++:
    Built as dynamic libs?:      YES
    C++ Compiler:                /usr/bin/c++  (ver 4.6.3)
    C++ flags (Release):         -fsigned-char -W -Wall -Werror=return-type -Werror=non-virtual-dtor -Werror=address -Werror=sequence-point -Wformat -Werror=format-security -Wmissing-declarations -Wundef -Winit-self -Wpointer-arith -Wshadow -Wsign-promo -fdiagnostics-show-option -Wno-long-long -pthread -fomit-frame-pointer -msse -msse2 -mno-avx -msse3 -mno-ssse3 -mno-sse4.1 -mno-sse4.2 -ffunction-sections -fvisibility=hidden -fvisibility-inlines-hidden -fopenmp -O3 -DNDEBUG  -DNDEBUG
    C++ flags (Debug):        

## Training
---

In the cells below, first some methods to generate the Noise needed for the GAN is defined, then 

In [10]:
'''Utilities'''
def trim(video):
    start = np.random.randint(0, video.shape[1] - (T+1))
    end = start + T
    return video[:, start:end, :, :]

# for input noises to generate fake video
# note that noises are trimmed randomly from n_frames to T for efficiency
def trim_noise(noise):
    #print("-----TRIMMING NOISE-----")
    #print(f"Noise Size: {noise.size()}")
    start = np.random.randint(0, noise.size(1) - (T+1))
    end = start + T
    #print("-----END OF TRIMMING NOISE-----")
    return noise[:, start:end, :, :, :]


''' calc grad of models '''

def bp_i(inputs, y, retain=False):
    label = (torch.FloatTensor()).cuda()
    label.resize_(inputs.size(0)).fill_(y)
    labelv = Variable(label)
    outputs = dis_i(inputs)
    err = criterion(outputs, labelv)
    err.backward(retain_graph=retain)
    toReturnErr = err.data[0] if err.size() == torch.Tensor().size() else err.item()
    return toReturnErr, outputs.data.mean()

def bp_v(inputs, y, retain=False):
    #print("----BackPropagate_V-----")
    #print(inputs.size())
    label = (torch.FloatTensor()).cuda()
    try:
        label.resize_(inputs.size(0)).fill_(y)

    except RuntimeError as _:
        # Dimension of y does not allow to use fill_
        assert(inputs.size(0) == y.size(0))
        label = (torch.FloatTensor(y)).cuda()

    labelv = Variable(label)
    outputs = dis_v(inputs)
    err = criterion(outputs, labelv)
    err.backward(retain_graph=retain)
    toReturnErr = err.data[0] if err.size() == torch.Tensor().size() else err.item()
    #print("----End of BackPropagate_V-----")
    return toReturnErr, outputs.data.mean()


''' gen input noise for fake video '''

def gen_z(n_frames, batch_size = batch_size):
    #print("----Generating Z-----")
    #print(f"N_FRAMES: {n_frames}")
    #print(f"BATCH_SIZE: {batch_size}")
    #print(f"D_C: {d_C}")
    #print(f"D_E: {d_E}")
    #print(f"nz: {nz}")
    z_C = Variable(torch.randn(batch_size, d_C))
    #  repeat z_C to (batch_size, n_frames, d_C)
    z_C = z_C.unsqueeze(1).repeat(1, n_frames, 1)
    eps = Variable(torch.randn(batch_size, d_E))
    if cuda == True:
        z_C, eps = z_C.cuda(), eps.cuda()

    gru.initHidden(batch_size)
    # notice that 1st dim of gru outputs is seq_len, 2nd is batch_size
    z_M = gru(eps, n_frames).transpose(1, 0)
    z = torch.cat((z_M, z_C), 2)  # z.size() => (batch_size, n_frames, nz)
    #print("----End Generating Z-----")
    return z.view(batch_size, n_frames, nz, 1, 1)

''' prepare for train '''
def timeSince(since):
    now = time.time()
    s = now - since
    d = math.floor(s / ((60**2)*24))
    h = math.floor(s / (60**2)) - d*24
    m = math.floor(s / 60) - h*60 - d*24*60
    s = s - m*60 - h*(60**2) - d*24*(60**2)
    return '%dd %dh %dm %ds' % (d, h, m, s)

trained_path = os.path.join(current_path, "mocogan", 'trained_models')
def checkpoint(model, optimizer, epoch):
    filename = os.path.join(trained_path, '%s_epoch-%d' % (model.__class__.__name__, epoch))
    torch.save(model.state_dict(), filename + '.model')
    torch.save(optimizer.state_dict(), filename + '.state')

def save_video(fake_video, epoch, runtimeError = False):
    outputdata = fake_video * 255
    outputdata = outputdata.astype(np.uint8)
    dir_path = os.path.join(current_path, 'mocogan', 'generated_videos')
    file_path = os.path.join(dir_path, 'fakeVideo_epoch-%d-%s.mp4' % (epoch, "RuntimeError" if runtimeError else ""))
    skvideo.io.vwrite(file_path, outputdata)


In [11]:
''' train models '''
def train():
    
    start_time = time.time()

    print(f"Starting training: CUDA is { 'On' if cuda == True else 'Off'}")

    for epoch in range(1, n_iter+1):
        ''' prepare real images '''
        # real_videos.size() => (batch_size, nc, T, img_size, img_size)

        # Get data iterator
        data_iter = iter(dataloader) #Iterator
        data_len = len(dataloader) #Num Batches
        data_i = 0

        processedClass = None

        while data_i < data_len:

            try:
                #batch_size = 16
                (real_videos, labels) = next(data_iter) #random_choice()

                ''' Process 1 video for each class while testing. '''
                #if (labels in processedClass):
                #    continue

                #else:
                #    processedClass.append(labels)
                ''' Process only 1 video class'''
                #if processedClass is None:
                #    processedClass = labels.item()
                #else:
                #    if processedClass != labels.item():
                #        continue

                for (key, val) in dictClassesIdx.items():
                    if ( val in labels.tolist() ):
                        pass
                        #print(key)

                if cuda == True:
                    real_videos = real_videos.cuda()

                real_videos = Variable(real_videos)
                real_img = real_videos[:, :, np.random.randint(0, T), :, :]

                ''' prepare fake images '''
                # note that n_frames is sampled from video length distribution
                n_frames = T + 2 + np.random.randint(0, real_videos.size()[2]) #video_lengths[np.random.randint(0, n_videos)]
                Z = gen_z(n_frames, batch_size)  # Z.size() => (batch_size, n_frames, nz, 1, 1)
                # trim => (batch_size, T, nz, 1, 1)
                Z = trim_noise(Z)
                # generate videos
                Z = Z.contiguous().view(batch_size*T, nz, 1, 1)
                fake_videos = gen_i(Z)
                fake_videos = fake_videos.view(batch_size, T, nc, img_size, img_size)
                # transpose => (batch_size, nc, T, img_size, img_size)
                fake_videos = fake_videos.transpose(2, 1)
                # img sampling
                fake_img = fake_videos[:, :, np.random.randint(0, T), :, :]

                ''' train discriminators '''
                # video
                dis_v.zero_grad()
                randomStartFrameIdx = np.random.randint(0, real_videos.size()[2] - T - 1)
                #print("-----INFOS-----")
                #print(f"RandomStartFrame:{randomStartFrameIdx}")
                #print(f"Video Size:{real_videos.size()}")
                #print("-----END OF INFOS-----")
                croppedRealVideos = real_videos[:,:,randomStartFrameIdx: randomStartFrameIdx + T, :, :]
                #err_Dv_real, Dv_real_mean = bp_v(croppedRealVideos, 0.9)
                err_Dv_real, Dv_real_mean = bp_v(croppedRealVideos, labels.type(torch.FloatTensor) / len(dictClassesIdx))
                err_Dv_fake, Dv_fake_mean = bp_v(fake_videos.detach(), 0)
                err_Dv = err_Dv_real + err_Dv_fake
                optim_Dv.step()
                # image
                dis_i.zero_grad()
                err_Di_real, Di_real_mean = bp_i(real_img, 0.9)
                err_Di_fake, Di_fake_mean = bp_i(fake_img.detach(), 0)
                err_Di = err_Di_real + err_Di_fake
                optim_Di.step()


                ''' train generators '''
                gen_i.zero_grad()
                gru.zero_grad()
                # video. notice retain=True for back prop twice
                err_Gv, _ = bp_v(fake_videos, 0.9, retain=True)
                # images
                err_Gi, _ = bp_i(fake_img, 0.9)
                optim_Gi.step()
                optim_GRU.step()

                '''Increment index for Batch'''
                data_i = data_i + 1
            except StopIteration:
                break
          
            except RuntimeError:
                print("Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.")
                epoch = epoch - 1
                save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch, True)
                break
          
            except KeyboardInterrupt:
                save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch)
                checkpoint(dis_i, optim_Di, epoch)
                checkpoint(dis_v, optim_Dv, epoch)
                checkpoint(gen_i, optim_Gi, epoch)
                checkpoint(gru,   optim_GRU, epoch)


        if epoch % n_epochs_display == 0:
            print('[%d/%d] (%s) Loss_Di: %.4f Loss_Dv: %.4f Loss_Gi: %.4f Loss_Gv: %.4f Di_real_mean %.4f Di_fake_mean %.4f Dv_real_mean %.4f Dv_fake_mean %.4f'
                  % (epoch, n_iter, timeSince(start_time), err_Di, err_Dv, err_Gi, err_Gv, Di_real_mean, Di_fake_mean, Dv_real_mean, Dv_fake_mean))

        if epoch % n_epochs_saveV == 0:
            save_video(fake_videos[0].data.cpu().numpy().transpose(1, 2, 3, 0), epoch)

        if epoch % n_epochs_check == 0:
            checkpoint(dis_i, optim_Di, epoch)
            checkpoint(dis_v, optim_Dv, epoch)
            checkpoint(gen_i, optim_Gi, epoch)
            checkpoint(gru,   optim_GRU, epoch)

## Loading Previous State
---
If wanted, the following cell can be used to load the previous state of a trained model.

In [12]:
''' use pre-trained models '''

def load():
    dis_i.load_state_dict(torch.load(trained_path + '/Discriminator_I.model'))
    dis_v.load_state_dict(torch.load(trained_path + '/Discriminator_V.model'))
    gen_i.load_state_dict(torch.load(trained_path + '/Generator_I.model'))
    gru.load_state_dict(torch.load(trained_path + '/GRU.model'))
    optim_Di.load_state_dict(torch.load(trained_path + '/Discriminator_I.state'))
    optim_Dv.load_state_dict(torch.load(trained_path + '/Discriminator_V.state'))
    optim_Gi.load_state_dict(torch.load(trained_path + '/Generator_I.state'))
    optim_GRU.load_state_dict(torch.load(trained_path + '/GRU.state'))

## Finally, start training


In [ ]:
load()
train()

Starting training: CUDA is On
[1/120000] (0d 0h 12m 56s) Loss_Di: 0.5370 Loss_Dv: 0.6628 Loss_Gi: 2.8111 Loss_Gv: 3.6682 Di_real_mean 0.6619 Di_fake_mean 0.0081 Dv_real_mean 0.5586 Dv_fake_mean 0.0275
Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.
[1/120000] (0d 0h 13m 9s) Loss_Di: 0.6135 Loss_Dv: 0.6477 Loss_Gi: 4.0602 Loss_Gv: 3.8416 Di_real_mean 0.9703 Di_fake_mean 0.1787 Dv_real_mean 0.5033 Dv_fake_mean 0.0174
Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.
[2/120000] (0d 0h 20m 59s) Loss_Di: 0.4095 Loss_Dv: 0.6575 Loss_Gi: 3.2490 Loss_Gv: 3.0582 Di_real_mean 0.9023 Di_fake_mean 0.0521 Dv_real_mean 0.4654 Dv_fake_mean 0.0335
Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.
[3/120000] (0d 0h 22m 35s) Loss_Di: 0.4623 Loss_Dv: 0.6143 Loss_Gi: 2.6

Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.
[28/120000] (0d 1h 55m 57s) Loss_Di: 0.4283 Loss_Dv: 0.6676 Loss_Gi: 3.0008 Loss_Gv: 4.5849 Di_real_mean 0.8487 Di_fake_mean 0.0395 Dv_real_mean 0.4923 Dv_fake_mean 0.0224
Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.
[29/120000] (0d 1h 57m 18s) Loss_Di: 0.5321 Loss_Dv: 0.6264 Loss_Gi: 2.6851 Loss_Gv: 4.3746 Di_real_mean 0.6793 Di_fake_mean 0.0159 Dv_real_mean 0.4624 Dv_fake_mean 0.0183
Encountered a Runtime error. Not Solved by pytorch community, maybe is a problem with Scikit-video library. Restarting the loop.
[30/120000] (0d 2h 0m 37s) Loss_Di: 0.4296 Loss_Dv: 0.6202 Loss_Gi: 3.1150 Loss_Gv: 3.9602 Di_real_mean 0.8205 Di_fake_mean 0.0246 Dv_real_mean 0.4762 Dv_fake_mean 0.0337
